In [1]:
#!/usr/bin/env python
# coding: utf-8

import pandas as pd
import numpy as np
import os
import re
import json

#cwd = dir_path = os.path.dirname(os.path.realpath(__file__)) 
# bidsdir = os.path.join('%s/../bids'%(cwd))
cwd='/ZPOOL/data/projects/rf1-mbme-pilot/code'
bidsdir='/ZPOOL/data/projects/rf1-mbme-pilot/bids/'

print(bidsdir)

/ZPOOL/data/projects/rf1-mbme-pilot/bids/


In [28]:
# Read events files, find unique sub IDs

event_files=[os.path.join(root,f) for root,dirs,files in os.walk(bidsdir) for f in files if f.endswith('events.tsv')]
#print(event_files)

data=[]
for file in event_files:
	sub='sub-'+re.search('func/sub-(.*)_task',file).group(1)
	acq=re.search('_acq-(.*)_events',file).group(1)
	tmp_df=pd.read_csv(file,sep='\t')
#	print(tmp_df)
	if tmp_df.shape[0]>0:
		#print(sub,acq)
		tmp_df['sub']=sub
		tmp_df['acq']=acq
		data.append(tmp_df)
events_df=pd.concat(data)

print(events_df)

#print("Unique subs:")
#print(events_df['sub'].unique())


          onset  duration                  trial_type  response_time  \
0      4.177403  0.488014       guess_leftButton_face       0.488014   
1      8.393883  1.021105    outcome_stranger_neutral            NaN   
2     10.438208  0.493859  guess_rightButton_computer       0.493859   
3     15.505598  1.009990     outcome_computer_punish            NaN   
4     18.533678  0.381692  guess_rightButton_computer       0.381692   
..          ...       ...                         ...            ...   
103  415.732665  1.005120                miss_outcome            NaN   
104  418.741135  0.633491  guess_rightButton_computer       0.633491   
105  422.943785  1.005950    outcome_computer_neutral            NaN   
106  426.953275  0.422033  guess_rightButton_computer       0.422033   
107  430.305535  1.004500     outcome_computer_reward            NaN   

           sub     acq  
0    sub-10422  mb6me4  
1    sub-10422  mb6me4  
2    sub-10422  mb6me4  
3    sub-10422  mb6me4  
4    sub-1

In [30]:
# Find unique acquisitions for each sub & append relevant info to exclusions dataframe

data=[]
for sub in events_df['sub'].unique():
    #print(sub)
    for acq in events_df['acq'].unique():
        
        # Check for abs motion file (generated by run_motioneval.sh)
        fname='%s/../derivatives/fsl/mcflirt/%s/%s/_abs.rms'%(cwd,sub,acq)
        #print(fname)
        if os.path.exists(fname):
            absolute=np.loadtxt(fname)
            
            file="/ZPOOL/data/projects/rf1-mbme-pilot/derivatives/mriqc/%s/func/%s_task-sharedreward_acq-%s_echo-2_bold.json"%(sub,sub,acq)
            if os.path.exists(file):
                f = open(file)
            else:
                file="/ZPOOL/data/projects/rf1-mbme-pilot/derivatives/mriqc/%s/func/%s_task-sharedreward_acq-%s_bold.json"%(sub,sub,acq)
                if os.path.exists(file):
                    f = open(file)
                else:
                    print(file+" does not exist")
            
            # Returns JSON object as a dictionary
            FD = json.load(f)
            f.close
            FD=FD['fd_mean']
            #print(FD)
            # Confirm that this fd_mean value is the same as from MRIQC
            FD=np.loadtxt('%s/../derivatives/fsl/mcflirt/%s/%s/_rel_mean.rms'%(cwd,sub,acq))
            row=[sub,acq,events_df[(events_df['sub']==sub)&(events_df['acq']==acq)]['trial_type'].str.count('miss_decision').sum(),absolute.max(),FD]
            #row=[sub,acq,
            #    events_df[(events_df['sub']==sub)&(events_df['acq']==acq)]['trial_type'].str.count('miss_decision').sum(),
            #    FD]
            data.append(row)

# Generate exclusions df
exclusions_df=pd.DataFrame(data=data,columns=['sub','acq','TrialCount_misses','Max_Abs_motion','FD_mean'])
#print(exclusions_df)
exclusions_df['FD_exclusion']=exclusions_df['FD_mean']>0.5
exclusions_df['ABS_exclusion']=exclusions_df['Max_Abs_motion']>1.35
exclusions_df['Beh_TrialExclusion']=exclusions_df['TrialCount_misses']>27
#fd_df=exclusions_df[exclusions_df['FD_mean'] > 0.5]
print(exclusions_df)

           sub     acq  TrialCount_misses  Max_Abs_motion    FD_mean  \
0    sub-10422  mb6me4                  0        1.027630  0.0548118   
1    sub-10422  mb1me1                  1        0.642810  0.0503383   
2    sub-10422  mb3me4                  0        0.737704  0.0868264   
3    sub-10422  mb3me1                  0        0.940240  0.0886571   
4    sub-10422  mb1me4                  0        1.761860   0.100059   
..         ...     ...                ...             ...        ...   
366  sub-10125  mb6me1                  0        0.887747  0.0783694   
367  sub-10028  mb6me4                  1        1.152040  0.0614201   
368  sub-10028  mb1me1                  0        3.206020   0.140367   
369  sub-10028  mb3me4                  1        0.195320  0.0431269   
370  sub-10028  mb1me4                  0        0.150869  0.0242303   

     FD_exclusion  ABS_exclusion  Beh_TrialExclusion  
0           False          False               False  
1           False        

In [31]:
results=exclusions_df.groupby(by='sub').sum().reset_index().rename(columns={"TrialCount_misses": "TotalCount_misses"})
print(results)
results['Beh_TotalExclusion']=results['TotalCount_misses']>81
results=results[['sub','TotalCount_misses','Beh_TotalExclusion']]

            sub  TotalCount_misses  Max_Abs_motion  FD_exclusion  \
0     sub-10003                 54        8.897047             0   
1     sub-10007                  2       18.582200             0   
2     sub-10008                 10        7.139238             0   
3     sub-10010                  2        2.335954             0   
4     sub-10014                  5        6.512891             0   
..          ...                ...             ...           ...   
58  sub-10690sp                  9        6.949923             0   
59  sub-10691sp                  1       15.595864             0   
60  sub-10777sp                 22        5.762411             0   
61  sub-10803sp                  2        5.815230             0   
62    sub-12042                  0        3.319861             0   

    ABS_exclusion  Beh_TrialExclusion  
0               4                   1  
1               6                   0  
2               1                   0  
3               0      

/tmp/ipykernel_1266453/1065450899.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  results=exclusions_df.groupby(by='sub').sum().reset_index().rename(columns={"TrialCount_misses": "TotalCount_misses"})


In [32]:
exclusions_df.merge(results,on='sub')
exclusions_df.to_csv('../derivatives/exclusions.csv', index=False)